In [1]:
import cv2
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import joblib
import numpy as np

In [2]:
dark_video = cv2.VideoCapture('data/img/dark.mov')
light_video = cv2.VideoCapture('data/img/light.mov')

In [3]:
def get_frame(name, vid, sec):
    vid.set(cv2.CAP_PROP_POS_MSEC, sec*1000)
    hasFrames, image = vid.read()
    if hasFrames:
        cv2.imwrite(name + str(count) + '.jpg', image)
    return hasFrames

In [4]:
frame_rate = 0.1
count = 1
sec = 0
success = get_frame("data/img/dark", dark_video, sec)
while success and count < 50:
    count += 1
    sec += frame_rate
    sec = round(sec, 2)
    success = get_frame("data/img/dark", dark_video, sec)
    
frame_rate = 0.1
count = 1
sec = 0
success = get_frame("data/img/light", light_video, sec)
while success and count < 50:
    count += 1
    sec += frame_rate
    sec = round(sec, 2)
    success = get_frame("data/img/light", light_video, sec)

In [5]:
X_train = []
y_train = []
for i in range(1, 50):
    img_s = cv2.imread('data/img/dark' + str(i) + '.jpg', 0).flatten()
    img_n = cv2.imread('data/img/light' + str(i) + '.jpg', 0).flatten()
    X_train.append(img_s)
    y_train.append("Dark")
    X_train.append(img_n)
    y_train.append("Light")
X_train = np.array(X_train)
print(X_train)


[[  6  10   7 ...  13  13  16]
 [250 234 212 ... 110 112 111]
 [ 11  11   9 ...  13  14  16]
 ...
 [ 53  53  54 ... 136 135 134]
 [ 13  10   9 ...   1   0   0]
 [ 50  50  49 ... 177 189 188]]


In [6]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

joblib.dump(model, 'shade.joblib')

['shade.joblib']

In [7]:
tree.export_graphviz(model, 
                     out_file='shade.dot', 
                     class_names=['Dark', 'Light'], 
                     label='all',
                     rounded=True, 
                     filled=True)

In [ ]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 60)
cv2.namedWindow("preview")

while cap.isOpened():
    ret, frame = cap.read()
    
    if frame is not None:
        processed_frame = [frame.flatten()[0:921600]]

        prediction = model.predict(processed_frame)

        cv2.putText(frame, f"Shade: {prediction[0]}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("preview", frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break